# Chatbot

Simple chatbot.
- Use Gradio to provide a chat interface
- Include some multi-shot promting
- Provide additional context to the llm during the conversation

In [3]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-'):
    print("API key looks good so far")
else:
    print("There might be a problem with your API key?")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [6]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [7]:
# Gradio takes a function called "chat" and will generate a chat interface
# The whole conversation history is passed in each time to give full context to the llm

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


History is:
[]
And messages is:
[{'role': 'system', 'content': "You are a helpful assistant in a clothes store. You should try to gently encourage the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. For example, if the customer says 'I'm looking to buy a hat', you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'Encourage the customer to buy hats if they are unsure what to get."}, {'role': 'user', 'content': 'Hi there'}]
History is:
[{'role': 'user', 'metadata': None, 'content': 'Hi there', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Hello! Welcome to our store! How can I assist you today? Are you looking for something specific?', 'options': None}]
And messages is:
[{'role': 'system', 'content': "You are a helpful assistant in a clothes store. You should try to gently encourage the customer to try items that are on sale. Hats are 60% off, and most oth

In [12]:
hats_to_get_rid_of = [
    {"item": "fireman's helmet", "price": "$10"},
    {"item": "old grubby baseball cap", "price": "$100"},
]
shoes_to_get_keep = [
    {"item": "2nd hand sandels", "price": "$50"},
    {"item": "Nike runners", "price": "$30"},
]


def chat(message, history):

    extra_context = system_message
    if "hat" in message:
        extra_context += f"If hats are mentioned we have old stock to shift. Try really hard to sale these hats, don't take no for an answer: {hats_to_get_rid_of}"

    if "shoes" in message:
        extra_context += f"If shoes are mentioned, actively discourage selling these {shoes_to_get_keep}"

    messages = [{"role": "system", "content": extra_context}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
